# Config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/third_take/src

/content/drive/MyDrive/Colab Notebooks/third_take/src


In [ ]:
#===========================================================
# Config
#===========================================================
import yaml

with open('./config.yaml') as file:
    config = yaml.safe_load(file.read())

config

{'cat_negative_value': True,
 'cm_class': ['0', '1'],
 'custom_eval': None,
 'debug': False,
 'do_log_scale': True,
 'do_weight': False,
 'feval': None,
 'fname_cm': 'cm.png',
 'fname_importance': 'feature_importance.png',
 'fname_log_cv': 'log_cv',
 'fname_log_fold': 'log_fold',
 'fname_log_pp': 'log_pp',
 'fname_submission': 'submission.csv',
 'id': 'object_id',
 'input_dir': './src/input/',
 'input_dir_jn': '../src/input/',
 'input_dir_root': './input/',
 'input_dir_root_jn': '../input/',
 'model': 'lgbm',
 'num_class': 1,
 'num_folds': 5,
 'num_seed': 1,
 'optim_th': False,
 'output_dir': './src/output/',
 'output_dir_cv': './src/output/',
 'output_dir_fold': '../src/input/',
 'output_dir_jn': '../src/output/',
 'output_dir_pp': '../src/input/',
 'seed': 1996,
 'shuffle': False,
 'split_group': 'art_series_id',
 'split_target': 'likes',
 'split_type': 'group',
 'target': 'likes',
 'task': 'regression'}

In [ ]:
#===========================================================
# Config
#===========================================================

df_path_dict = {
    'train': config['input_dir_root_jn']+'train.csv',
    'test': config['input_dir_root_jn']+'test.csv',
    'sample_submission': config['input_dir_root_jn']+'sample_submission.csv',
    'folds': config['input_dir_jn']+'folds.csv',
}

In [ ]:
config['feat_excluded'] = [
]

# Library

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!pip install fastText
!rm -rf fastText

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 6.79 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
Checking out files: 100% (526/526), done.
     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3098169 sha256=ff1eddb486defe73c1bc3e7ac9bcba0406fdd2d0fdfe0d228ada20a28dc640d6
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fastText


In [ ]:
from fasttext import load_model

# パスを書き換えてください
model = load_model("../bin/lid.176.bin")

# model.predict("Arcadian Landscape with Shepherds and Cattle")
# model.predict("De schilder H.W. Mesdag voor een doek")

## BERT Library

In [ ]:
BERT_PRETRAINED_MODEL_ARCHIVE_LIST = [
    "bert-base-uncased",
    "bert-large-uncased",
    "bert-base-cased",
    "bert-large-cased",
    "bert-base-multilingual-uncased",
    "bert-base-multilingual-cased",
    "bert-base-chinese",
    "bert-base-german-cased",
    "bert-large-uncased-whole-word-masking",
    "bert-large-cased-whole-word-masking",
    "bert-large-uncased-whole-word-masking-finetuned-squad",
    "bert-large-cased-whole-word-masking-finetuned-squad",
    "bert-base-cased-finetuned-mrpc",
    "bert-base-german-dbmdz-cased",
    "bert-base-german-dbmdz-uncased",
    "cl-tohoku/bert-base-japanese",
    "cl-tohoku/bert-base-japanese-whole-word-masking",
    "cl-tohoku/bert-base-japanese-char",
    "cl-tohoku/bert-base-japanese-char-whole-word-masking",
    "TurkuNLP/bert-base-finnish-cased-v1",
    "TurkuNLP/bert-base-finnish-uncased-v1",
    "wietsedv/bert-base-dutch-cased",
    "Geotrend/bert-base-fr-cased",
    "Geotrend/bert-base-es-cased", 
    "Geotrend/bert-base-it-cased", 
    # See all BERT models at https://huggingface.co/models?filter=bert
]




# transformers というライブラリに入ってる BERT を使用します
!pip install transformers > /dev/null


import pandas as pd
import numpy as np
import torch
import transformers

from transformers import BertTokenizer
from tqdm import tqdm
tqdm.pandas()


class BertSequenceVectorizer:
    def __init__(self, model_name = 'bert-base-uncased'):
        assert model_name in BERT_PRETRAINED_MODEL_ARCHIVE_LIST
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules neologdn
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules mojimoji
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules xgboost
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules texthero
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules ptitprince
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules gensim
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules catboost
# !pip install --target /content/drive/MyDrive/Colab\ Notebooks/my_modules transformers > /dev/null
# ### Colabと Notebooks の間に space が入っているので、
# ### \ (backslash)でエスケープしておかないとエラーになります。
# # Step 3: Python に自分専用モジュール置場の場所を教えてあげる（毎回）
# ## ここまで準備が整ったら、
# import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/my_modules')

In [ ]:
#!pip install lightgbm
!pip install mojimoji
!pip install neologdn
!pip install xgboost
!pip install catboost
!pip install gensim
# !pip install MeCab
!pip install ptitprince
!pip install texthero

     |████████████████████████████████| 133kB 5.2MB/s 
     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for neologdn: filename=neologdn-0.4-cp37-cp37m-linux_x86_64.whl size=188240 sha256=a67ae24e63baefff9a755f2082870c52fb662c72103113b642c158551f93a6f1
  Stored in directory: /root/.cache/pip/wheels/b3/6f/d4/c132b4e7aef22019e307e7673d97010644c9c15f28c0d0b018
Successfully built neologdn
     |████████████████████████████████| 65.7MB 106kB/s 
     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for ptitprince: filename=ptitprince-0.2.5-cp37-none-any.whl size=8429 sha256=8c08f18bed43d385cb1d6c4b691f669f8b5b5a20ab6c331060f111dd3b1ca4b5
  Stored in directory: /root/.cache/pip/wheels/fb/c1/75/2ef8da161053bdf8e03f9a360f426accd4fac1ddf7f6ed0029
Successfully built ptitprince
     |████████████████████████████████| 1.4MB 7.8MB/s 
     |████████████████████████████████| 245kB 22.6MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434673 s

In [ ]:
#===========================================================
# Library
#===========================================================

import gc
import itertools
import json
import os
import random
import sys
import time
import warnings
from collections import Counter, defaultdict
from contextlib import contextmanager
from functools import partial
from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

warnings.filterwarnings("ignore")

import builtins
import types

import lightgbm as lgb
import matplotlib.pyplot as plt
#import MeCab
import mojimoji
import neologdn
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import torch
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from gensim.models.word2vec import Word2Vec
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import NMF, PCA, TruncatedSVD
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer,
                                             _document_frequency)
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import (GroupKFold, GroupShuffleSplit, KFold,
                                     StratifiedKFold)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.validation import check_is_fitted
from tqdm import tqdm
from PIL import ImageColor



from pathlib import Path

from gensim.models import word2vec, KeyedVectors
from tqdm import tqdm

import texthero as hero
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

import nltk

nltk.download('stopwords')
os.listdir(os.path.expanduser('~/nltk_data/corpora/stopwords/'))

class AbstractBaseBlock:
    def fit(self, input_df: pd.DataFrame, y=None):
        return self.transform(input_df)
    
    def transform(self, input_df: pd.DataFrame) -> pd.DataFrame:
        raise NotImplementedError()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pd.set_option('display.max_columns', 2000)

# Utils

In [ ]:
#===========================================================
# Utils
#===========================================================

def seed_everything(seed=1996):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


@contextmanager
def timer(name):
    t0 = time.time()
    logger.info(f'[{name}] start')
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')
    logger.info('')


def get_logger(filename='log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log", mode='w')
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = get_logger(config['output_dir_jn']+config['fname_log_pp'])

def load_df(path, df_name, config):
    if path.split('.')[-1]=='csv':
        if config['debug']:
            df = pd.read_csv(path, nrows=1000)
        else:
            df = pd.read_csv(path)
    elif path.split('.')[-1]=='pkl':
        df = pd.read_pickle(path)
    logger.info(f"{df_name} shape / {df.shape} ")
    return df

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        logger.info('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df



def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )




# https://github.com/nyk510/vivid/blob/master/vivid/utils.py

def decorate(s: str, decoration=None):
    if decoration is None:
        decoration = '★' * 20
        
    return ' '.join([decoration, str(s), decoration])

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        if self.verbose is None:
            return
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)
    

def run_blocks(input_df, blocks, y=None, test=False):
    out_df = pd.DataFrame()
    
    print(decorate('start run blocks...'))

    with Timer(prefix='run test={}'.format(test)):
        for block in feature_blocks:
            with Timer(prefix='\t- {}'.format(str(block))):
                if not test:
                    out_i = block.fit(input_df, y=y)
                else:
                    out_i = block.transform(input_df)

            assert len(input_df) == len(out_i), block
            name = block.__class__.__name__
            out_df = pd.concat([out_df, out_i.add_suffix(f'_{name}')], axis=1)
        
    return out_df

## Taeget Encoding

In [ ]:
#===========================================================
# Target Encoding
#===========================================================

def target_encoding(train_fold, valid_fold, test_fold, key_columns):
    print('Target encoding')

    te_columns = [
        
    ]

    for key in key_columns:
        for col in te_columns:

            train_fold[key +'_' + col + '_te'] = np.nan
            group = LabelEncoder().fit_transform(train_fold['PlaceID'])
            skf = GroupKFold(n_splits=3)

            splits = skf.split(train_fold, train_fold['AverageLandPrice'], group)
            for tr_idx, vl_idx in splits:
                tr = train_fold.iloc[tr_idx]
                vl = train_fold.iloc[vl_idx]

                te = tr.groupby([key])[col].mean()

                train_fold[key +'_' + col + '_te'].iloc[vl_idx] = vl[key].map(te)

            te = train_fold.groupby([key])[col].mean()

            valid_fold[key +'_' + col + '_te'] = valid_fold[key].map(te)
            test_fold[key +'_' + col + '_te'] = test_fold[key].map(te)

    return train_fold, valid_fold, test_fold

## Text Processer

In [ ]:
#===========================================================
# Text Processer
#===========================================================

class BM25Transformer(BaseEstimator, TransformerMixin):
    """
    Parameters
    ----------
    use_idf : boolean, optional (default=True)
    k1 : float, optional (default=2.0)
    b  : float, optional (default=0.75)
    References
    ----------
    Okapi BM25: a non-binary model - Introduction to Information Retrieval
    http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
    """
    def __init__(self, use_idf=True, k1=2.0, b=0.75):
        self.use_idf = use_idf
        self.k1 = k1
        self.b = b

    def fit(self, X):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features] document-term matrix
        """
        if not sp.sparse.issparse(X):
            X = sp.sparse.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)
            idf = np.log((n_samples - df + 0.5) / (df + 0.5))
            self._idf_diag = sp.sparse.spdiags(idf, diags=0, m=n_features, n=n_features)

        doc_len = X.sum(axis=1)
        self._average_document_len = np.average(doc_len)

        return self

    def transform(self, X, copy=True):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features] document-term matrix
        copy : boolean, optional (default=True)
        """
        if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
            # preserve float family dtype
            X = sp.sparse.csr_matrix(X, copy=copy)
        else:
            # convert counts or binary occurrences to floats
            X = sp.sparse.csr_matrix(X, dtype=np.float, copy=copy)

        n_samples, n_features = X.shape

        # Document length (number of terms) in each row
        # Shape is (n_samples, 1)
        doc_len = X.sum(axis=1)
        # Number of non-zero elements in each row
        # Shape is (n_samples, )
        sz = X.indptr[1:] - X.indptr[0:-1]

        # In each row, repeat `doc_len` for `sz` times
        # Shape is (sum(sz), )
        # Example
        # -------
        # dl = [4, 5, 6]
        # sz = [1, 2, 3]
        # rep = [4, 5, 5, 6, 6, 6]
        rep = np.repeat(np.asarray(doc_len), sz)

        # Compute BM25 score only for non-zero elements
        nom = self.k1 + 1
        denom = X.data + self.k1 * (1 - self.b + self.b * rep / self._average_document_len)
        data = X.data * nom / denom

        X = sp.sparse.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

        if self.use_idf:
            check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

            expected_n_features = self._idf_diag.shape[0]
            if n_features != expected_n_features:
                raise ValueError("Input has n_features=%d while the model"
                                 " has been trained with n_features=%d" % (
                                     n_features, expected_n_features))
            X = X * self._idf_diag

        return X

def normalize_text(text):
    text = mojimoji.zen_to_han(text, kana=False)
    text = neologdn.normalize(text)
    return text

def tokenize(text):
    return_tokens = []
    tokenizer = MeCab.Tagger()
    tokenizer.parse("")
    text = normalize_text(text)
    tokens = tokenizer.parse(text)
    tokens = tokens.split("\n")
    for token_info in tokens:
        token = token_info.split("\t")[0]
        if token == "EOS":
            return_tokens = " ".join(return_tokens)
            return return_tokens
        else:
            pos = token_info.split("\t")[1]
            pos = pos.split(",")
            part = pos[0]
            if part == "記号":
                if token not in ["．", "。"]:
                    continue
                return_tokens.append(token)
            elif part in ["形容詞", "動詞", "形容動詞"]:
                return_tokens.append(pos[-3])
            elif part in ["名詞", "副詞", "感動詞", "助詞"]:
                return_tokens.append(token)

def tfidf_svd(df, cols, n_components=16, language="en", seed=1996):
    return_df = []
    for col in cols:
        print(f'[tfidf_svd] generating features from: {col}')
        text_df = df[col].fillna("NULL")
        if language == "ja":
            text_df = text_df.map(tokenize)

        tfv = TfidfVectorizer(min_df=2,  max_features=None,
                            strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                            ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
        svd = TruncatedSVD(
            n_components=n_components, random_state=seed)
        
        tfidf_result = tfv.fit_transform(text_df)
        svd_result = svd.fit_transform(tfidf_result)
        svd_result = pd.DataFrame(svd_result)
        svd_result = svd_result.add_prefix('TFIDF_SVD_{}_'.format(col))
        return_df.append(svd_result)
    
    return_df = pd.concat(return_df, axis=1)
    return return_df


def tfidf_nmf(df, cols, n_components=16, language="en", seed=1996):
    return_df = []
    for col in cols:
        print(f'[tfidf_nmf] generating features from: {col}')
        text_df = df[col].fillna("NULL")
        if language == "ja":
            text_df = text_df.map(tokenize)

        tfv = TfidfVectorizer(min_df=2,  max_features=None,
                            strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                            ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
        nmf = NMF(
            n_components=n_components, random_state=seed, verbose=1
        )
        
        tfidf_result = tfv.fit_transform(text_df)
        nmf_result = nmf.fit_transform(tfidf_result)
        nmf_result = pd.DataFrame(nmf_result)
        nmf_result = nmf_result.add_prefix('TFIDF_NMF_{}_'.format(col))
        return_df.append(nmf_result)
    
    return_df = pd.concat(return_df, axis=1)
    return return_df

def tfidf_bm25_svd(df, cols, n_components=16, language="en", seed=1996):
    return_df = []
    for col in cols:
        print(f'[tfidf_bm25_svd] generating features from: {col}')
        text_df = df[col].fillna("NULL")
        if language == "ja":
            text_df = text_df.map(tokenize)

        tfv = TfidfVectorizer(min_df=2,  max_features=None,
                            strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                            ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
        bm25 = BM25Transformer(use_idf=True, k1=2.0, b=0.75)
        svd = TruncatedSVD(
            n_components=n_components, random_state=seed)
        
        tfidf_result = tfv.fit_transform(text_df)
        
        bm25.fit(tfidf_result)
        bm25_result = bm25.transform(tfidf_result)

        svd_result = svd.fit_transform(bm25_result)
        svd_result = pd.DataFrame(svd_result)
        svd_result = svd_result.add_prefix('TFIDF_BM25_SVD_{}_'.format(col))
        return_df.append(svd_result)
    
    return_df = pd.concat(return_df, axis=1)
    return return_df



In [ ]:
##textの長さをはかる
def create_string_length_feature(input_df, columns):
    out_df = pd.DataFrame()

    str_columns = [
        'title', 
        'long_title',
        'sub_title',
        'more_title',
        'description'
    ]
    for column in columns:
      assert column in str_columns

    for c in columns:
        out_df[c] = input_df[c].str.len()

    return out_df.add_prefix('StringLength__')

In [ ]:
def text_normalization(text):
    
    # 英語とオランダ語を stopword として指定
    custom_stopwords = nltk.corpus.stopwords.words('dutch') + nltk.corpus.stopwords.words('english')

    x = hero.clean(text, pipeline=[
        hero.preprocessing.fillna,
        hero.preprocessing.lowercase,
        hero.preprocessing.remove_digits,
        hero.preprocessing.remove_punctuation,
        hero.preprocessing.remove_diacritics,
        lambda x: hero.preprocessing.remove_stopwords(x, stopwords=custom_stopwords)
    ])
    
    return x

class TfidfBlock(AbstractBaseBlock):
    """tfidf x SVD による圧縮を行なう block"""
    def __init__(self, column: str):
        """
        args:
            column: str
                変換対象のカラム名
        """
        self.column = column
    
    def preprocess(self, input_df):
        x = text_normalization(input_df[self.column])
        return x
    
    def get_master(self, input_df):
        """tdidfを計算するための全体集合を返す. 
        デフォルトでは fit でわたされた dataframe を使うが, もっと別のデータを使うのも考えられる."""
        return input_df
    
    def fit(self, input_df, y=None):
        master_df = self.get_master(input_df)
        text = self.preprocess(input_df)
        self.pileline_ = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=10000)),
            ('svd', TruncatedSVD(n_components=50)),
        ])
        
        self.pileline_.fit(text)
        return self.transform(input_df)
    
    def transform(self, input_df):
        text = self.preprocess(input_df)
        z = self.pileline_.transform(text)
        
        out_df = pd.DataFrame(z)
        return out_df.add_prefix(f'{self.column}_tfidf_')

In [ ]:
class BM25Block(AbstractBaseBlock):
    """tfidf x SVD による圧縮を行なう block"""
    def __init__(self, column: str):
        """
        args:
            column: str
                変換対象のカラム名
        """
        self.column = column
    
    def preprocess(self, input_df):
        x = text_normalization(input_df[self.column])
        return x
    
    def get_master(self, input_df):
        """tdidfを計算するための全体集合を返す. 
        デフォルトでは fit でわたされた dataframe を使うが, もっと別のデータを使うのも考えられる."""
        return input_df
    
    def fit(self, input_df, y=None):
        master_df = self.get_master(input_df)
        text = self.preprocess(input_df)
        self.pileline_ = Pipeline([
            ('tfidf', TfidfVectorizer(min_df=2,  max_features=None,
                            strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                            ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)),
            ('BM25', BM25Transformer(use_idf=True, k1=2.0, b=0.75)),
            ('svd',TruncatedSVD(n_components=50, random_state=1996)),
        ])
        
        self.pileline_.fit(text)
        return self.transform(input_df)
    
    def transform(self, input_df):
        text = self.preprocess(input_df)
        z = self.pileline_.transform(text)
        
        out_df = pd.DataFrame(z)
        return out_df.add_prefix(f'{self.column}_BM25_')

In [ ]:
# palette = pd.read_csv('../input/palette.csv')

# palette_max = palette.loc[palette.groupby('object_id')['ratio'].idxmax(),:][["object_id","ratio", "color_r", "color_g", "color_b"]]
# #palette_max.columns = ['object_id'] + [c + "_argmax" for c in palette_max.columns[1:]]

## 実験スペース

In [ ]:
technique = pd.read_csv("../input/technique.csv")
material = pd.read_csv("../input/material.csv")
collection = pd.read_csv("../input/object_collection.csv")

In [ ]:
mat_col = pd.concat([material, collection], axis=0).reset_index(drop=True)
mat_tec = pd.concat([material, technique], axis=0).reset_index(drop=True)
col_tec = pd.concat([collection, technique], axis=0).reset_index(drop=True)
mat_col_tec = pd.concat([material, collection, technique], axis=0).reset_index(drop=True)

In [ ]:
mat_col_tec.groupby("object_id")["name"].apply(list)

object_id
000405d9a5e3f49fc49d       [photographic paper, cardboard, albumen print]
001020bd00b149970f78                [oil paint (paint), panel, paintings]
0011d6be41612ec9eae3               [oil paint (paint), canvas, paintings]
0012765f7a97ccc3e9e9    [photographic paper, salted paper print, album...
00133be3ff222c9b74b0                             [paper, prints, etching]
                                              ...                        
fff4bbb55fd7702d294e                  [photographic paper, albumen print]
fffbe07b997bec00e203       [photographic paper, cardboard, albumen print]
fffd1675758205748d7f                                      [albumen print]
fffd43b134ba7197d890                  [photographic paper, albumen print]
ffff22ea12d7f99cff31           [paper, photographic paper, albumen print]
Name: name, Length: 23953, dtype: object

In [ ]:
# 単語ベクトル表現の次元数
# 元の語彙数をベースに適当に決めました
model_size = {
    "material": 20,
    "technique": 8,
    "collection": 3,
    "material_collection": 20,
    "material_technique": 20,
    "collection_technique": 10,
    "material_collection_technique": 25
}

n_iter = 100

In [ ]:
w2v_dfs = []
for df, df_name in zip(
        [
            material, collection, technique,
            mat_col, mat_tec, col_tec, mat_col_tec
        ], [
            "material", "collection", "technique",
            "material_collection",
            "material_technique",
            "collection_technique",
            "material_collection_technique"
        ]):
    df_group = df.groupby("object_id")["name"].apply(list).reset_index()
    # Word2Vecの学習
    w2v_model = word2vec.Word2Vec(df_group["name"].values.tolist(),
                                  size=model_size[df_name],
                                  min_count=1,
                                  window=1,
                                  iter=n_iter)

    # 各文章ごとにそれぞれの単語をベクトル表現に直し、平均をとって文章ベクトルにする
    sentence_vectors = df_group["name"].progress_apply(
        lambda x: np.mean([w2v_model.wv[e] for e in x], axis=0))
    sentence_vectors = np.vstack([x for x in sentence_vectors])
    sentence_vector_df = pd.DataFrame(sentence_vectors,
                                      columns=[f"{df_name}_w2v_{i}"
                                               for i in range(model_size[df_name])])
    sentence_vector_df.index = df_group["object_id"]
    w2v_dfs.append(sentence_vector_df)

100%|██████████| 23953/23953 [00:00<00:00, 32645.99it/s]


In [ ]:
# for w2v_df in w2v_dfs:
#   df = df.merge(w2v_df, on='object_id', how='left')

## Merge_df

In [ ]:
def merge_df(df):
    # information of maker and production
    #maker = pd.read_csv('../input/maker.csv')
    principal_maker = pd.read_csv('../input/principal_maker.csv')
    principal_maker_occupation = pd.read_csv('../input/principal_maker_occupation.csv')
    
    principal_maker_occupation = principal_maker_occupation.pivot_table(
        index='id',columns='name',aggfunc=len, fill_value=0
    ).reset_index()
    
    principal_maker_qualification = principal_maker.pivot_table(
        index='object_id',columns='qualification',aggfunc='size', fill_value=0
    ).reset_index()
    principal_maker_qualification.columns = ['object_id'] + [c+'_qualification' for c in principal_maker_qualification.columns[1:]]
    
    principal_maker_roles = principal_maker.pivot_table(
        index='object_id',columns='roles',aggfunc='size', fill_value=0
    ).reset_index()
    principal_maker_roles.columns =['object_id'] +  [c+'_roles' for c in principal_maker_roles.columns[1:]]

    principal_maker_productionPlaces = principal_maker.pivot_table(
        index='object_id',columns='productionPlaces',aggfunc='size', fill_value=0
    ).reset_index()
    principal_maker_productionPlaces.columns = ['object_id'] +  [c+'_productionPlaces' for c in principal_maker_productionPlaces.columns[1:]]

    #principal_maker_maker_name = principal_maker.pivot_table(
    #    index='object_id',columns='maker_name',aggfunc='size', fill_value=0
    #).reset_index()

    principal_maker = principal_maker.merge(principal_maker_occupation, on='id', how='left')

    occupation_col = [
        'auctioneer', 'bookseller', 'designer', 'draughtsman',
        'goldsmith', 'instrument maker', 'jeweler', 'painter', 'print maker',
        'printer', 'publisher', 'sculptor'
    ]

    principal_maker[occupation_col] = principal_maker[occupation_col].fillna(0).astype(int)

    occupation_cnt= principal_maker.groupby(['object_id'])[occupation_col].agg('sum').reset_index()
    occupation_cnt.columns = ['object_id'] + [c+'_occupation' for c in occupation_cnt.columns[1:]]
    
    df = df.merge(principal_maker_qualification, on='object_id', how='left')
    df = df.merge(principal_maker_roles, on='object_id', how='left')
    df = df.merge(principal_maker_productionPlaces, on='object_id', how='left')
    df = df.merge(occupation_cnt, on='object_id', how='left')
    
    #========================================================================
    
    #color
    color = pd.read_csv('../input/color.csv')
    #color = color[color["percentage"]>10].reset_index(drop = True)
    color = pd.concat([color, pd.DataFrame(color['hex'].str.strip().map(ImageColor.getrgb).values.tolist(), columns=['R', 'G', 'B'])],axis = 1)
    

    color_max = color.loc[color.groupby('object_id')['percentage'].idxmax(),:][["object_id", "R", "G", "B"]]
    #color_max.columns = ['object_id'] + [c + "_argmax" for c in color_max.columns[1:]]

    # color["sa"] = np.zeros(len(color))
    # for index, co in color.iterrows():
    #   iter = color_max[color_max["object_id"] == co["object_id"]]
    #   #me = (((iter[["color_r", "color_g", "color_b"]] - pa[["color_r", "color_g", "color_b"]])**2).mean(axis = 1))**0.5
    #   me = np.linalg.norm(iter[["R", "G", "B"]] - co[["R", "G", "B"]])
    #   color["sa"][index] = me
    #   # print(me)
    #   # if index ==100:
    #   #   break
    # color_sa = color.groupby(["object_id"])["sa"].mean()
    # df = df.merge(color_sa, on='object_id', how='left') 


    #color_max = color.loc[color.groupby('object_id')['percentage'].idxmax(),:][["object_id", "R", "G", "B"]]
    color_max.columns = ['object_id'] + [c + "_argmax" for c in color_max.columns[1:]]

    df = df.merge(color_max, on='object_id', how='left')    

    color['times_color_R_percentage'] = color['R'] * color['percentage']/100
    color['times_color_G_percentage'] = color['G'] * color['percentage']/100
    color['times_color_B_percentage'] = color['B'] * color['percentage']/100
    
    color_gb = color.groupby(['object_id'])['R', 'G', 'B'].agg(['median', 'mean', 'std', 'min', 'max']).reset_index()
    color_gb.columns = ['object_id'] + [c1+'_'+c2 for c1, c2 in color_gb.columns[1:]]
    df = df.merge(color_gb, on='object_id', how='left')
    
    # color_weighted_gb = color.groupby(['object_id'])['times_color_R_percentage', 'times_color_G_percentage', 'times_color_B_percentage'].agg(['median', 'mean', 'std', 'min', 'max']).reset_index()
    # color_weighted_gb.columns = ['object_id'] + [c1+'_'+c2 for c1, c2 in color_weighted_gb.columns[1:]]
    # df = df.merge(color_weighted_gb, on='object_id', how='left')

    
    # historical_person
    # historical_person = pd.read_csv('../input/historical_person.csv')
    
    # material
    material = pd.read_csv('../input/material.csv')
    material_name = material.pivot_table(
        index='object_id',columns='name',aggfunc='size', fill_value=0
    ).reset_index()
    material_name.columns = ['object_id'] + [c+'_material' for c in material_name.columns[1:]]
    df = df.merge(material_name, on='object_id', how='left')
    
    # object_collection
    object_collection = pd.read_csv('../input/object_collection.csv')
    object_collection_name = object_collection.pivot_table(
        index='object_id',columns='name',aggfunc='size', fill_value=0
    ).reset_index()
    object_collection_name.columns = ['object_id'] + [c+'_object_collection' for c in object_collection_name.columns[1:]]
    df = df.merge(object_collection_name, on='object_id', how='left')
    
    # palette
    palette = pd.read_csv('../input/palette.csv')
    #palette = palette[palette['ratio'] > 0.05].reset_index(drop = True)


    palette_max = palette.loc[palette.groupby('object_id')['ratio'].idxmax(),:][["object_id", "color_r", "color_g", "color_b"]]
    palette_max.columns = ['object_id'] + [c + "_argmax" for c in palette_max.columns[1:]]
    
    df = df.merge(palette_max, on='object_id', how='left') 




    palette['times_color_r_raito'] = palette['color_r'] * palette['ratio']
    palette['times_color_g_raito'] = palette['color_g'] * palette['ratio']
    palette['times_color_b_raito'] = palette['color_b'] * palette['ratio']
    
    palette_gb = palette.groupby(['object_id'])['color_g', 'color_r', 'color_b'].agg(['median', 'mean', 'std', 'min', 'max']).reset_index()
    palette_gb.columns = ['object_id'] + [c1+'_'+c2 for c1, c2 in palette_gb.columns[1:]]
    df = df.merge(palette_gb, on='object_id', how='left')
    
    # palette_weighted_gb = palette.groupby(['object_id'])['times_color_r_raito', 'times_color_g_raito', 'times_color_b_raito'].agg(['median', 'mean', 'std', 'min', 'max']).reset_index()
    # palette_weighted_gb.columns = ['object_id'] + [c1+'_'+c2 for c1, c2 in palette_weighted_gb.columns[1:]]
    # df = df.merge(palette_weighted_gb, on='object_id', how='left')
    
    # production_place
    #production_place = pd.read_csv('../input/production_place.csv')
    #production_place['name'] = production_place['name'].map(lambda x: x.replace('? ', ''))
    #production_place_name = production_place.pivot_table(
    #    index='object_id',columns='name',aggfunc='size', fill_value=0
    #).reset_index()
    #df = df.merge(production_place_name, on='object_id', how='left')
    
    # technique
    technique = pd.read_csv('../input/technique.csv')
    technique_name = technique.pivot_table(
        index='object_id',columns='name',aggfunc='size', fill_value=0
    ).reset_index()
    technique_name.columns = ['object_id'] + [c+'_technique' for c in technique_name.columns[1:]]
    df = df.merge(technique_name, on='object_id', how='left')
    
    return df

#  Main

In [ ]:
logger.info('Basic Config')
for k, v in config.items():
    logger.info('   {}: {}'.format(k, v))
logger.info('')

Basic Config
   debug: False
   task: regression
   input_dir_root: ./input/
   input_dir_root_jn: ../input/
   input_dir: ./src/input/
   input_dir_jn: ../src/input/
   output_dir: ./src/output/
   output_dir_jn: ../src/output/
   output_dir_fold: ../src/input/
   output_dir_pp: ../src/input/
   output_dir_cv: ./src/output/
   fname_submission: submission.csv
   fname_log_fold: log_fold
   fname_log_pp: log_pp
   fname_log_cv: log_cv
   fname_importance: feature_importance.png
   fname_cm: cm.png
   cm_class: ['0', '1']
   id: object_id
   target: likes
   seed: 1996
   num_folds: 5
   split_type: group
   split_group: art_series_id
   split_target: likes
   shuffle: False
   model: lgbm
   num_class: 1
   num_seed: 1
   do_log_scale: True
   cat_negative_value: True
   do_weight: False
   custom_eval: None
   feval: None
   optim_th: False
   feat_excluded: []



In [ ]:
with timer('Data Loading'):
    train = load_df(path=df_path_dict['train'], df_name='train', config=config)
    test = load_df(path=df_path_dict['test'], df_name='test', config=config)
    sample_submission = load_df(path=df_path_dict['sample_submission'], df_name='sample_submission', config=config)
    folds = load_df(path=df_path_dict['folds'], df_name='folds', config=config)
    gc.collect()

[Data Loading] start
train shape / (12026, 19) 
test shape / (12008, 18) 
sample_submission shape / (12008, 1) 
folds shape / (12026, 3) 
[Data Loading] done in 2 s



In [ ]:
tr_len = len(train)
te_len = len(test)

In [ ]:
train.head()

,object_id,art_series_id,title,description,long_title,principal_maker,principal_or_first_maker,sub_title,copyright_holder,more_title,acquisition_method,acquisition_date,acquisition_credit_line,dating_presenting_date,dating_sorting_date,dating_period,dating_year_early,dating_year_late,likes
0,0011d6be41612ec9eae3,93c092ba70beab248f31,The Avenue of Birches,NaN,"The Avenue of Birches, Jan Hackaert, 1660 - 1685",Jan Hackaert,Jan Hackaert,h 66.5cm × w 53.7cm × t 2.5cm × d 4.7cm,NaN,The Avenue of Birches,purchase,1808-01-01T00:00:00,NaN,1660 - 1685,1660.0,17,1660.0,1685.0,48
1,0012765f7a97ccc3e9e9,95c14fb11c54281ad7e0,Struik in bloei,NaN,"Struik in bloei, Richard Tepe (attributed to),...",Richard Tepe,Richard Tepe,h 165mm × w 223mm,erven Richard Tepe,Struik in bloei,purchase,2000-01-01T00:00:00,NaN,c. 1900 - c. 1930,1900.0,19,1900.0,1930.0,2
2,0017be8caa87206532cb,4c406890d208fe01f8fb,Portret van een vrouw,"Portret van eenvrouw, zittend naast een tafel.","Portret van een vrouw, Tresling & Comp., 1860 ...",Tresling & Comp.,Tresling & Comp.,h 87mm × w 56mm,NaN,Portret van een vrouw,gift,2007-01-01T00:00:00,"Gift of M.M. Boom, Leiden",1860 - 1880,1860.0,19,1860.0,1880.0,5
3,00181d86ff1a7b95864e,fa0891535500a4973db2,A St Bernard Dog,"Een sint-bernardshond, staand in een landschap...","A St Bernard Dog, Bernard te Gempt, c. 1850 - ...",Bernard te Gempt,Bernard te Gempt,h 179cm × w 248cm × t 4cm,NaN,A St Bernard Dog,bequest,1881-01-01T00:00:00,"J. Hilman Bequest, Amsterdam",c. 1850 - c. 1879,1850.0,19,1850.0,1879.0,100
4,001c52ae28ec106d9cd5,8065ed5063c5887e677d,Woelige zee met zeilschepen,Woelige zee met zeilschepen.,"Woelige zee met zeilschepen, anonymous, 1825 -...",anonymous,anonymous,h 13cm × w 17.5cm × d 0.7cm,NaN,Woelige zee met zeilschepen,unknown,1971-01-01T00:00:00,NaN,1825 - 1874,1825.0,19,1825.0,1874.0,7


In [ ]:
config['feat_excluded'] += [
    'art_series_id', 
    'title', 'description', 'long_title', 'more_title', 
    'acquisition_date', 'dating_presenting_date',
]

## Merge optional data

In [ ]:
train = merge_df(train)
test = merge_df(test)

In [ ]:
for w2v_df in w2v_dfs:
  train = train.merge(w2v_df, on='object_id', how='left')
  test = test.merge(w2v_df, on='object_id', how='left')

## sub title

In [ ]:
train['sub_title']

0        h 66.5cm × w 53.7cm × t 2.5cm × d 4.7cm
1                              h 165mm × w 223mm
2                                h 87mm × w 56mm
3                      h 179cm × w 248cm × t 4cm
4                    h 13cm × w 17.5cm × d 0.7cm
                          ...                   
12021                           h 90mm × w 140mm
12022                          h 245mm × w 278mm
12023                          h 217mm × w 306mm
12024                          h 560mm × w 405mm
12025               h 50.5cm × w 43.5cm × d 14cm
Name: sub_title, Length: 12026, dtype: object

In [ ]:
for axis in ['h', 'w', 't', 'd']:
    column_name = f'size_{axis}'
    size_info = train['sub_title'].str.extract(r'{} (\d*|\d*\.\d*)(cm|mm)'.format(axis)) # 正規表現を使ってサイズを抽出
    size_info = size_info.rename(columns={0: column_name, 1: 'unit'})
    size_info[column_name] = size_info[column_name].replace('', np.nan).astype(float) # dtypeがobjectになってるのでfloatに直す
    size_info[column_name] = size_info.apply(lambda row: row[column_name] * 10 if row['unit'] == 'cm' else row[column_name], axis=1) # 　単位をmmに統一する
    train[column_name] = size_info[column_name] # trainにくっつける
for axis in ['h', 'w', 't', 'd']:
    column_name = f'size_{axis}'
    size_info = test['sub_title'].str.extract(r'{} (\d*|\d*\.\d*)(cm|mm)'.format(axis)) # 正規表現を使ってサイズを抽出
    size_info = size_info.rename(columns={0: column_name, 1: 'unit'})
    size_info[column_name] = size_info[column_name].replace('', np.nan).astype(float) # dtypeがobjectになってるのでfloatに直す
    size_info[column_name] = size_info.apply(lambda row: row[column_name] * 10 if row['unit'] == 'cm' else row[column_name], axis=1) # 　単位をmmに統一する
    test[column_name] = size_info[column_name] # trainにくっつける

train_sub_title = train[['size_h', 'size_w', 'size_t', 'size_d']].agg(['max', 'min', 'std', 'mean', 'median'], axis=1)
test_sub_title = test[['size_h', 'size_w', 'size_t', 'size_d']].agg(['max', 'min', 'std', 'mean', 'median'], axis=1)
train_sub_title['max_min_sa'] = train_sub_title['max'] - train_sub_title['min']
train_sub_title['max_min_hi'] = train_sub_title['max']/train_sub_title['min']
test_sub_title['max_min_sa'] = train_sub_title['max'] - train_sub_title['min']
test_sub_title['max_min_hi'] = train_sub_title['max']/train_sub_title['min']
train_sub_title.columns = ['sub_title_' + c for c in train_sub_title.columns]
test_sub_title.columns = ['sub_title_' + c for c in test_sub_title.columns]
train = pd.concat([train, train_sub_title], axis = 1)
test = pd.concat([test, test_sub_title], axis = 1)

In [ ]:
config['feat_excluded'] += ['sub_title']

## data feature

In [ ]:
def get_acquisition_date_feature(df):
    df_copy = df.copy()
    df_copy['acquisition_date'] = pd.to_datetime(df_copy['acquisition_date'])
    df['acquisition_date_year'] = df_copy['acquisition_date'].dt.year
    return df

In [ ]:
train = get_acquisition_date_feature(train)
test = get_acquisition_date_feature(test)

In [ ]:
def get_dating_presenting_date_featre(df):
    df_copy = df.copy()
    df_copy['dating_presenting_date'] = df_copy['dating_presenting_date'].fillna('None').map(lambda x:x.replace('c. ', '').split('-'))
    df_copy['dating_presenting_date'][df_copy['dating_presenting_date'].map(lambda x:len(x) == 1)] =\
    df_copy['dating_presenting_date'][df_copy['dating_presenting_date'].map(lambda x:len(x) == 1)].map(lambda x:x.append(x[0]))
    df['dating_presenting_date_start'] = df_copy['dating_presenting_date'].map(lambda x:x[0])
    df['dating_presenting_date_end'] = df_copy['dating_presenting_date'].map(lambda x:x[1])
    
    df['dating_presenting_date_start'][df['dating_presenting_date_start'].map(lambda x:'before' in x)] =\
    df['dating_presenting_date_start'][df['dating_presenting_date_start'].map(lambda x:'before' in x)].map(lambda x:x.split()[-1])
    df['dating_presenting_date_start'][df['dating_presenting_date_start'].map(lambda x:'after' in x)] =  np.nan
    df['dating_presenting_date_start'] [df['dating_presenting_date_start'] == 'None'] = np.nan
    
    df['dating_presenting_date_end'][df['dating_presenting_date_end'].map(lambda x:'after' in x)] =\
    df['dating_presenting_date_end'][df['dating_presenting_date_end'].map(lambda x:'after' in x)].map(lambda x:x.split()[-1])
    df['dating_presenting_date_end'][df['dating_presenting_date_end'].map(lambda x:'before' in x)] =  np.nan
    df['dating_presenting_date_end'] [df['dating_presenting_date_end'] == 'None'] = np.nan
    
    df['dating_presenting_date_end'] = df['dating_presenting_date_end'].map(float)
    df['dating_presenting_date_start'] = df['dating_presenting_date_start'].map(float)
    
    df['dating_presenting_date_range'] = df['dating_presenting_date_end'] - df['dating_presenting_date_start']

    return df

In [ ]:
train = get_dating_presenting_date_featre(train)
test = get_dating_presenting_date_featre(test)

## text feature

In [ ]:
text_col = ['title', 'description', 'long_title', 'more_title']

In [ ]:
train_strength = create_string_length_feature(train, text_col)
test_strength = create_string_length_feature(test, text_col)
train = pd.concat([train, train_strength], axis=1)
test = pd.concat([test, test_strength], axis=1)

In [ ]:
train["title_lang_ft"] = train["title"].fillna("").map(
    lambda x: model.predict(x.replace("\n", ""))[0][0])
test["title_lang_ft"] = test["title"].fillna("").map(
    lambda x: model.predict(x.replace("\n", ""))[0][0])

In [ ]:
df = train.append(test, sort=False)

In [ ]:
# with timer('tfidf_svd'):
#     df_text = tfidf_svd(df, text_col, n_components=64, seed=config['seed'])
#     train_text = df_text[:len(train)].reset_index(drop=True)
#     test_text = df_text[len(train):].reset_index(drop=True)

#     train = pd.concat([train, train_text], axis=1)
#     test = pd.concat([test, test_text], axis=1)

In [ ]:
# with timer('tfidf_bm25_svd'):
#     df_text = tfidf_bm25_svd(df, text_col, n_components=64, seed=config['seed'])
#     train_text = df_text[:len(train)].reset_index(drop=True)
#     test_text = df_text[len(train):].reset_index(drop=True)

#     train = pd.concat([train, train_text], axis=1)
#     test = pd.concat([test, test_text], axis=1)

### atma講座の書き方

In [ ]:
feature_blocks = [
    *[TfidfBlock(c) for c in text_col],
    *[BM25Block(c) for c in text_col],
]

In [ ]:
with timer('tfidf_BM25i_svd'):
    df_text = run_blocks(df, blocks=feature_blocks)
    train_text = df_text[:len(train)].reset_index(drop=True)
    test_text = df_text[len(train):].reset_index(drop=True)

    train = pd.concat([train, train_text], axis=1)
    test = pd.concat([test, test_text], axis=1)

[tfidf_BM25i_svd] start


★★★★★★★★★★★★★★★★★★★★ start run blocks... ★★★★★★★★★★★★★★★★★★★★
	- <__main__.TfidfBlock object at 0x7f1a495ff5d0> 4.327[s]
	- <__main__.TfidfBlock object at 0x7f1a495ff390> 11.418[s]
	- <__main__.TfidfBlock object at 0x7f1a495ffc10> 6.287[s]
	- <__main__.TfidfBlock object at 0x7f1a4e1bd0d0> 6.071[s]
	- <__main__.BM25Block object at 0x7f1a4e1bdfd0> 5.387[s]
	- <__main__.BM25Block object at 0x7f1a4e1bdcd0> 15.792[s]
	- <__main__.BM25Block object at 0x7f1a4e1bd3d0> 8.511[s]


[tfidf_BM25i_svd] done in 66 s



	- <__main__.BM25Block object at 0x7f1a4e1bd6d0> 8.209[s]
run test=False 66.290[s]


In [ ]:
train.columns

Index(['object_id', 'art_series_id', 'title', 'description', 'long_title',
       'principal_maker', 'principal_or_first_maker', 'sub_title',
       'copyright_holder', 'more_title',
       ...
       'more_title_BM25_40_BM25Block', 'more_title_BM25_41_BM25Block',
       'more_title_BM25_42_BM25Block', 'more_title_BM25_43_BM25Block',
       'more_title_BM25_44_BM25Block', 'more_title_BM25_45_BM25Block',
       'more_title_BM25_46_BM25Block', 'more_title_BM25_47_BM25Block',
       'more_title_BM25_48_BM25Block', 'more_title_BM25_49_BM25Block'],
      dtype='object', length=842)

### BERT

In [ ]:
def bert_svd(df, cols, n_components=16, model_name = "bert-base-uncased", seed=1996):
    return_df = []
    for col in cols:
        print(f'[bert_svd] generating features from: {col}')
        bert_result = df[col].fillna("NULL")


        BSV = BertSequenceVectorizer(model_name) # インスタンス化します

        svd = TruncatedSVD(
            n_components=n_components, random_state=seed)

        bert_result = bert_result.progress_apply(lambda x: BSV.vectorize(x))
        bert_result = pd.DataFrame(np.stack(bert_result))
            

        svd_result = svd.fit_transform(bert_result)
        svd_result = pd.DataFrame(svd_result)
        svd_result = svd_result.add_prefix('bert_SVD_{}_'.format(col))
        return_df.append(svd_result)
    
    return_df = pd.concat(return_df, axis=1)
    return return_df

In [ ]:
#オランダ、英語、 ドイツ、フランス、スペイン、イタリア
bert_language = ["wietsedv/bert-base-dutch-cased", "bert-base-cased", 
                 "bert-base-german-dbmdz-cased", "Geotrend/bert-base-fr-cased",
                 "Geotrend/bert-base-es-cased", "Geotrend/bert-base-it-cased", "bert-base-multilingual-cased"]
#オランダ、英語、 ドイツ、フランス、スペイン、イタリア

language_List = ["__label__nl", "__label__en", "__label__de", "__label__fr", "__label__es", "__label__it"]

In [ ]:
'''こっちは, second以降で使ったやつ'''
with timer('bert_svd'):
  df_langs = []
  for i, model_n in enumerate(bert_language):
    if i != len(bert_language)-1:
      df_lang = df[df["title_lang_ft"] == language_List[i]]
      df_text = bert_svd(df_lang, text_col, n_components=64, model_name = model_n, seed=config['seed'])
      df_lang = df_lang.reset_index(drop = True)[["object_id"]]
      df_lang = pd.concat([df_lang, df_text], axis = 1)
      df_langs.append(df_lang)
      #print(df_lang)
    else:
      #@をつけると, 変数を扱える
      df_lang = df.query('title_lang_ft not in @language_List')
      df_text = bert_svd(df_lang, text_col, n_components=64, model_name = model_n, seed=config['seed'])
      df_lang = df_lang.reset_index(drop = True)[["object_id"]]
      df_lang = pd.concat([df_lang, df_text], axis = 1)
      df_langs.append(df_lang)


  '''secondでのbertのでーたの書き出し'''
for i in range(len(df_langs)df_langs[-1].to_csv("bert_svd_multilanguage.csv")):
  df_langs[i].to_csv("../model_temp/bert_svd_{}.csv".format(language_List[i]))
df_langs[-1].to_csv("../model_temp/bert_svd_multilanguage.csv")


In [ ]:
# '''second以降でのbertのデータ読み込み'''
# df_langs = []
# for i in range(len(language_List)):
#   df_langs.append(pd.read_csv("../model_temp/bert_svd_{}.csv".format(language_List[i])))
# df_langs.append(pd.read_csv("../model_temp/bert_svd_multilanguage.csv"))

In [ ]:
# for i in df_langs:
#   print(i.shape)

In [ ]:
# with timer('bert_svd'):
#     df_text = bert_svd(df, text_col, n_components=64, seed=config['seed'])
#     train_text = df_text[:len(train)].reset_index(drop=True)
#     test_text = df_text[len(train):].reset_index(drop=True)

#     train = pd.concat([train, train_text], axis=1)
#     test = pd.concat([test, test_text], axis=1)

# df_text.to_csv("bert_bert_vec.csv")


# df_text = pd.read_csv("../src/bert_vec.csv")
# train_text = df_text[:len(train)].reset_index(drop=True)
# test_text = df_text[len(train):].reset_index(drop=True)

# train = pd.concat([train, train_text], axis=1)
# test = pd.concat([test, test_text], axis=1)

In [ ]:
'''second以降で使うconcat'''
df_langs_concat = pd.concat(df_langs)
train = train.merge(df_langs_concat, how = 'left', on = 'object_id')
test = test.merge(df_langs_concat, how = 'left', on = 'object_id')

In [ ]:
# train_kari.loc[:, train_kari.columns.str.startswith('bert')]

In [ ]:
'''copasta_bertのデータ読み込み'''
copa_bert = pd.read_csv("../model_temp/bert_vecs.csv")
copa_bert.columns = ['copasta_' + c for c in copa_bert.columns[:-1]] + ['object_id']

In [ ]:
train = train.merge(copa_bert, how='left', on='object_id')
test = test.merge(copa_bert, how='left', on='object_id')

In [ ]:
copa_Xml = pd.read_csv("../model_temp/xlm_roberta_vecs.csv")

In [ ]:
train = train.merge(copa_Xml, how='left', on='object_id')
test = test.merge(copa_Xml, how='left', on='object_id')

In [ ]:
# with timer('bert_svd'):
#     df_text = bert_svd(df, text_col, n_components=64, model_name = 'wietsedv/bert-base-dutch-cased', seed=config['seed'])
#     train_text = df_text[:len(train)].reset_index(drop=True)
#     test_text = df_text[len(train):].reset_index(drop=True)

#     train = pd.concat([train, train_text], axis=1)
#     test = pd.concat([test, test_text], axis=1)

In [ ]:
# df["title_lang_ft"].value_counts()

## Category feature

In [ ]:
obj_col = ['principal_maker', 'principal_or_first_maker', 'copyright_holder', 'acquisition_method', 'acquisition_credit_line', 'title_lang_ft']

In [ ]:
cnt_df = train.append(test, sort=False).reset_index(drop=True)

In [ ]:
print('Count encoding')
ce_col = []
for col in obj_col:
        te = cnt_df[col].value_counts()
        train[col+'_count'] = train[col].map(te)
        test[col+'_count'] = test[col].map(te)
        ce_col.append(col+'_count')

Count encoding


## paletteのembedding

In [ ]:
embedding_df = pd.read_pickle('../model_temp/palette_embedding999.pkl')
embedding_df[['object_id']] = embedding_df.index
train = train.merge(embedding_df, on = 'object_id', how = 'left')
test = test.merge(embedding_df, on = 'object_id', how = 'left')

## 画像のCNN

In [ ]:
df_train_predict = pd.read_pickle('../model_temp/cnn_train_predict.pkl')
df_test_predict = pd.read_pickle('../model_temp/cnn_test_predict.pkl')
train = train.merge(df_train_predict, on = 'object_id', how = 'left')
test = test.merge(df_test_predict, on = 'object_id', how = 'left')

# 分割

In [ ]:
assert len(train) == tr_len
assert len(test) == te_len

In [ ]:
categorical_col = []
for col in test.columns:

    if col in config['feat_excluded'] :
        continue

    if col not in df.columns:
        continue 
        
    if df[col].dtype != 'object':
        continue

    categorical_col.append(col)
categorical_col    

['object_id',
 'principal_maker',
 'principal_or_first_maker',
 'copyright_holder',
 'acquisition_method',
 'acquisition_credit_line',
 'title_lang_ft']

In [ ]:
with timer('Data Processing'):
    for cv in range(5):
        #if cv in [0, 1, 2, 3]:
        #    continue
        print('Start fold:', cv)

        train_idx, valid_idx = folds[folds['folds'] != cv].index,  folds[folds['folds'] == cv].index
        print('train:', len(train_idx), ', valid:', len(valid_idx))
        
        train_fold = train.iloc[train_idx].reset_index(drop=True)
        valid_fold = train.iloc[valid_idx].reset_index(drop=True)
        test_fold = test.copy()
        
        # train_fold, valid_fold, test_fold = target_encoding(train_fold, valid_fold, test_fold,
        #                                               ['category1-category2']
        #                                                  )

        train_fold, valid_fold, test_fold = target_encoding(train_fold, valid_fold, test_fold,
                                                      categorical_col
                                                         )
        
        
        
        print('label encoding')
        df_copy = df.copy()
        for col in test_fold.columns:

            if col in config['feat_excluded'] :
                continue

            if col not in df.columns:
                continue 
                
            if df[col].dtype != 'object':
                continue

            print(col)
            
            le = LabelEncoder()
            le.fit(df_copy[col].fillna('None').astype('category'))
            train_fold[col] = le.transform(train_fold[col].fillna('None').astype('category'))
            valid_fold[col] = le.transform(valid_fold[col].fillna('None').astype('category'))
            test_fold[col] = le.transform(test_fold[col].fillna('None').astype('category'))
        
        print(train_fold.shape, valid_fold.shape, test_fold.shape)
        #break
        
        logger.info('train: {}, valid: {}, test:{}'.format(train_fold.shape, valid_fold.shape, test_fold.shape))
        train_fold.to_pickle(config['output_dir_pp']+f'train_fold{cv}.pkl')
        valid_fold.to_pickle(config['output_dir_pp']+f'valid_fold{cv}.pkl')
        test_fold.to_pickle(config['output_dir_pp']+f'test_fold{cv}.pkl')
        
        print()

[Data Processing] start


Start fold: 0
train: 9620 , valid: 2406
Target encoding
label encoding
object_id
principal_maker
principal_or_first_maker
copyright_holder
acquisition_method


train: (9620, 1490), valid: (2406, 1490), test:(12008, 1489)


acquisition_credit_line
title_lang_ft
(9620, 1490) (2406, 1490) (12008, 1489)

Start fold: 1
train: 9621 , valid: 2405
Target encoding
label encoding
object_id


train: (9621, 1490), valid: (2405, 1490), test:(12008, 1489)


...

train: (9621, 1490), valid: (2405, 1490), test:(12008, 1489)


acquisition_method
acquisition_credit_line
title_lang_ft
(9621, 1490) (2405, 1490) (12008, 1489)

Start fold: 4
train: 9621 , valid: 2405
Target encoding
label encoding
object_id
principal_maker


train: (9621, 1490), valid: (2405, 1490), test:(12008, 1489)


principal_or_first_maker
copyright_holder
acquisition_method
acquisition_credit_line
title_lang_ft
(9621, 1490) (2405, 1490) (12008, 1489)


[Data Processing] done in 23 s



In [ ]:
config['feat_excluded']

['art_series_id',
 'title',
 'description',
 'long_title',
 'more_title',
 'acquisition_date',
 'dating_presenting_date',
 'sub_title']

In [ ]:
train_fold[[col for col in test_fold.columns if col not in config['feat_excluded']]]

object_id  principal_maker  principal_or_first_maker  copyright_holder  \
0             2             1782                      1793                23   
1             3             2880                      2900                40   
2             5             3063                      3083                23   
3             6              404                       406                23   
4             8             3246                      3268                23   
...         ...              ...                       ...               ...   
9616      24023              736                       739                23   
9617      24024               45                        45                23   
9618      24026             2574                      2591                23   
9619      24027             2905                      2925                23   
9620      24028              381                       383                23   

      acquisition_method  acquisition_credit_line  dating_sorting_date  \
0                      6                      495               1660.0   
1                      6                      495               1900.0   
2                      3                      328               1860.0   
3                      1                      457               1850.0   
4                      8                      495               1825.0   
...                  ...                      ...                  ...   
9616                   7                      495               1900.0   
9617                   6                      495               1701.0   
9618                   7                      495               1778.0   
9619                   6                      495               1689.0   
9620                   1                      471               1500.0   

      dating_period  dating_year_early  dating_year_late  after_qualification  \
0                17             1660.0            1685.0                  NaN   
1                19             1900.0            1930.0                  0.0   
2                19             1860.0            1880.0                  NaN   
3                19             1850.0            1879.0                  NaN   
4                19             1825.0            1874.0                  NaN   
...             ...                ...               ...                  ...   
9616             19             1900.0            1920.0                  NaN   
9617             18             1701.0            1714.0                  NaN   
9618             18             1778.0            1778.0                  0.0   
9619             17             1689.0            1690.0                  0.0   
9620             15             1500.0            1535.0                  0.0   

      attributed to_qualification  attributed to workshop of_qualification  \
0                             NaN                                      NaN   
1                             1.0                                      0.0   
2                             NaN                                      NaN   
3                             NaN                                      NaN   
4                             NaN                                      NaN   
...                           ...                                      ...   
9616                          NaN                                      NaN   
9617                          NaN                                      NaN   
9618                          0.0                                      0.0   
9619                          0.0                                      0.0   
9620                          0.0                                      0.0   

      circle of_qualification  copy after_qualification  \
0                         NaN                       NaN   
1                         0.0                       0.0   
2                         NaN                       NaN   
3         

In [ ]:
test_fold[[col for col in test_fold.columns if col not in config['feat_excluded']]]

object_id  principal_maker  principal_or_first_maker  copyright_holder  \
0              0              411                       413                23   
1              1             1738                      1748                23   
2              4             3013                      3033                23   
3              7             3049                      3069                42   
4             11             1806                      1817                23   
...          ...              ...                       ...               ...   
12003      24029             1443                      1452                23   
12004      24030             2097                      2112                23   
12005      24031              967                       971                23   
12006      24032             1443                      1452                23   
12007      24033             3246                      3268                23   

       acquisition_method  acquisition_credit_line  dating_sorting_date  \
0                       7                      495               1850.0   
1                       5                      495               1609.0   
2                       3                      262               1779.0   
3                       6                      495               1895.0   
4                       8                      495               1700.0   
...                   ...                      ...                  ...   
12003                   7                      495               1871.0   
12004                   7                      495               1851.0   
12005                   7                      495               1856.0   
12006                   7                      495               1871.0   
12007                   7                      495               1850.0   

       dating_period  dating_year_early  dating_year_late  \
0                 19             1850.0            1900.0   
1                 17             1609.0            1633.0   
2                 18             1779.0            1779.0   
3                 19             1895.0            1898.0   
4                 17             1700.0            1700.0   
...              ...                ...               ...   
12003             19             1871.0            1871.0   
12004             19             1851.0            1885.0   
12005             19             1856.0            1859.0   
12006             19             1871.0            1871.0   
12007             19             1850.0            1870.0   

       after_qualification  attributed to_qualification  \
0                      0.0                          0.0   
1                      0.0                          0.0   
2                      NaN                          NaN   
3                      NaN                          NaN   
4                      NaN                          NaN   
...                    ...                          ...   
12003                  NaN                          NaN   
12004                  0.0                          1.0   
12005                  NaN                          NaN   
12006                  NaN                          NaN   
12007                  NaN                          NaN   

       attributed to workshop of_qualification  circle of_qualification  \
0                                          0.0                      0.0   
1                                          0.0                      0.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
...                                        ...                      ...   
12003                                      NaN                      NaN   
12004                                      0.0                      0.0   
12005           

In [ ]:
train_fold.isnull().sum() / len(train_fold)

object_id             0.000000
art_series_id         0.000000
title                 0.000000
description           0.291030
long_title            0.000000
                        ...   
color_embedding_60    0.001767
color_embedding_61    0.001767
color_embedding_62    0.001767
color_embedding_63    0.001767
pred_likes            0.001767
Length: 1490, dtype: float64

In [ ]:
test_fold.isnull().sum() / len(test_fold)

object_id             0.000000
art_series_id         0.000000
title                 0.000000
description           0.306879
long_title            0.000000
                        ...   
color_embedding_60    0.001666
color_embedding_61    0.001666
color_embedding_62    0.001666
color_embedding_63    0.001666
pred_likes            0.001666
Length: 1489, dtype: float64

In [ ]:
for col in train_fold.columns:
    print(col, train_fold[col].nunique()/len(train_fold))

object_id 1.0
art_series_id 0.9286976405779025
title 0.8579149776530506
description 0.5805009874233448
long_title 0.9211100717181167
principal_maker 0.2014343623323979
principal_or_first_maker 0.20424072341752417
sub_title 0.7319405467207151
copyright_holder 0.0031181789834736516
more_title 0.8750649620621557
acquisition_method 0.0009354536950420954
acquisition_date 0.0665211516474379
acquisition_credit_line 0.038457540796175034
dating_presenting_date 0.25485916224924643
dating_sorting_date 0.04937116723833281
dating_period 0.0010393929944912171
dating_year_early 0.04937116723833281
dating_year_late 0.04905934933998545
likes 0.05571146450472924
after_qualification 0.00020787859889824343
attributed to_qualification 0.0003118178983473651
attributed to workshop of_qualification 0.00020787859889824343
circle of_qualification 0.00020787859889824343
copy after_qualification 0.00020787859889824343
falsification of_qualification 0.00010393929944912172
follower of_qualification 0.00020787859889

In [ ]:
train_fold[[col for col in test_fold.columns if col not in config['feat_excluded']]].columns

Index(['object_id', 'principal_maker', 'principal_or_first_maker',
       'copyright_holder', 'acquisition_method', 'acquisition_credit_line',
       'dating_sorting_date', 'dating_period', 'dating_year_early',
       'dating_year_late',
       ...
       'color_embedding_55', 'color_embedding_56', 'color_embedding_57',
       'color_embedding_58', 'color_embedding_59', 'color_embedding_60',
       'color_embedding_61', 'color_embedding_62', 'color_embedding_63',
       'pred_likes'],
      dtype='object', length=1481)

In [ ]:
test_fold[[col for col in train_fold.columns if col not in config['feat_excluded'] + ['likes']]].columns

Index(['object_id', 'principal_maker', 'principal_or_first_maker',
       'copyright_holder', 'acquisition_method', 'acquisition_credit_line',
       'dating_sorting_date', 'dating_period', 'dating_year_early',
       'dating_year_late',
       ...
       'color_embedding_55', 'color_embedding_56', 'color_embedding_57',
       'color_embedding_58', 'color_embedding_59', 'color_embedding_60',
       'color_embedding_61', 'color_embedding_62', 'color_embedding_63',
       'pred_likes'],
      dtype='object', length=1481)

In [ ]:
##監視システム

# !pip install hyperdash
# from hyperdash import monitor_cell
# !hyperdash signup --github

In [ ]:
# %%monitor_cell "test001"
# BSV = BertSequenceVectorizer() # インスタンス化します

# # train = pd.read_csv(CFG.TRAIN_PATH)
# train['description'] = train['description'].fillna("NaN") # null は代わりのもので埋めます
# train['description_feature'] = train['description'].progress_apply(lambda x: BSV.vectorize(x))
# train[['object_id', 'description', 'description_feature']].head()

# 実験スペース

## テキストの前処理練習

In [ ]:
# class AbstractBaseBlock:
#     def fit(self, input_df: pd.DataFrame, y=None):
#         return self.transform(input_df)
    
#     def transform(self, input_df: pd.DataFrame) -> pd.DataFrame:
#         raise NotImplementedError()

In [ ]:
# !pip install ptitprince
# !pip install texthero

# import texthero as hero
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.pipeline import Pipeline

# import nltk

# nltk.download('stopwords')
# os.listdir(os.path.expanduser('~/nltk_data/corpora/stopwords/'))

In [ ]:
# import texthero as hero
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.pipeline import Pipeline

# import nltk

# nltk.download('stopwords')
# os.listdir(os.path.expanduser('~/nltk_data/corpora/stopwords/'))

In [ ]:
# raw_text = train['description']

# clean_text = hero.clean(raw_text, pipeline=[
#     hero.preprocessing.fillna,
#     hero.preprocessing.lowercase,
#     hero.preprocessing.remove_digits,
#     hero.preprocessing.remove_punctuation,
#     hero.preprocessing.remove_diacritics,
#     hero.preprocessing.remove_stopwords
# ])

In [ ]:
# # これがこうなる
# i = 101
# print('{} \n-> {}'.format(raw_text.values[i], clean_text.values[i]))

In [ ]:
# hero.visualization.wordcloud(clean_text, colormap='viridis', background_color='white')

なるほどよくわかりませんが、短い単語 (de / en など) がいくつもあってなんとなく前置詞とかの意味をなさない単語のように見えますね。discussion でもあがっていましたが text はオランダ語である場合があります。オランダ語での stopword も考慮するように少し直します。

In [ ]:
# # 英語とオランダ語を stopword として指定
# custom_stopwords = nltk.corpus.stopwords.words('dutch') + nltk.corpus.stopwords.words('english')

# apply_stopword_text = hero.clean(raw_text, pipeline=[
#     hero.preprocessing.fillna,
#     hero.preprocessing.lowercase,
#     hero.preprocessing.remove_digits,
#     hero.preprocessing.remove_punctuation,
#     hero.preprocessing.remove_diacritics,
#     lambda x: hero.preprocessing.remove_stopwords(x, stopwords=custom_stopwords)
# ])

In [ ]:
# hero.visualization.wordcloud(apply_stopword_text, colormap='viridis', background_color='white')

僕はオランダ語わかりませんが portret (ポートレイト・肖像画) や foto (写真) などが見て取れます。先ほどより前置詞のような単語が表に出てこなくなっていることが確認できます。

In [ ]:
# def text_normalization(text):
    
#     # 英語とオランダ語を stopword として指定
#     custom_stopwords = nltk.corpus.stopwords.words('dutch') + nltk.corpus.stopwords.words('english')

#     x = hero.clean(text, pipeline=[
#         hero.preprocessing.fillna,
#         hero.preprocessing.lowercase,
#         hero.preprocessing.remove_digits,
#         hero.preprocessing.remove_punctuation,
#         hero.preprocessing.remove_diacritics,
#         lambda x: hero.preprocessing.remove_stopwords(x, stopwords=custom_stopwords)
#     ])
    
#     return x

# class TfidfBlock(AbstractBaseBlock):
#     """tfidf x SVD による圧縮を行なう block"""
#     def __init__(self, column: str):
#         """
#         args:
#             column: str
#                 変換対象のカラム名
#         """
#         self.column = column
    
#     def preprocess(self, input_df):
#         x = text_normalization(input_df[self.column])
#         return x
    
#     def get_master(self, input_df):
#         """tdidfを計算するための全体集合を返す. 
#         デフォルトでは fit でわたされた dataframe を使うが, もっと別のデータを使うのも考えられる."""
#         return input_df
    
#     def fit(self, input_df, y=None):
#         master_df = self.get_master(input_df)
#         text = self.preprocess(input_df)
#         self.pileline_ = Pipeline([
#             ('tfidf', TfidfVectorizer(max_features=10000)),
#             ('svd', TruncatedSVD(n_components=50)),
#         ])
        
#         self.pileline_.fit(text)
#         return self.transform(input_df)
    
#     def transform(self, input_df):
#         text = self.preprocess(input_df)
#         z = self.pileline_.transform(text)
        
#         out_df = pd.DataFrame(z)
#         return out_df.add_prefix(f'{self.column}_tfidf_')

In [ ]:
# block = TfidfBlock('title')

# block.fit(train)

# assert block.transform(train).equals(block.transform(train))
# assert block.transform(test).equals(block.transform(test))

In [ ]:
# block = TfidfBlock('title')

# block.fit(df)

# assert block.transform(train).equals(block.transform(train))
# assert block.transform(test).equals(block.transform(test))

## feature_blocksの使い方

In [ ]:
# feature_blocks = [
#     # one hot encoding
#     *[OneHotEncoding(c) for c in [
#         'title', 'description', 'long_title',
#        'principal_maker', 'principal_or_first_maker', 'sub_title',
#        'copyright_holder', 'more_title', 'acquisition_method',
#        'acquisition_date', 'acquisition_credit_line', 'dating_presenting_date',
#        'dating_sorting_date', 'dating_period', 'dating_year_early',
#        'dating_year_late',]],
    
#     # count encoding
#     *[CountEncodingBlock(c) for c in [
#         'art_series_id', 
#         'title', 'description', 'long_title',
#        'principal_maker', 'principal_or_first_maker', 'sub_title',
#        'copyright_holder', 'more_title', 'acquisition_method',
#        'acquisition_date', 'acquisition_credit_line', 'dating_presenting_date',
#        'dating_sorting_date', 'dating_period', 'dating_year_early',
#        'dating_year_late',]],
    
#     # 文字列長さ
#     *[StringLengthBlock(c) for c in [
#         'title', 'description', 'long_title',
#        'principal_maker', 'principal_or_first_maker', 'sub_title',
#     ]],
#     # historical person
#     HistoricalPersonBlock(),
    
#     # description の tfidf
#     TfidfBlock('description')
# ]

# train_feat_df = run_blocks(train_df, blocks=feature_blocks, test=False)

In [ ]:
'''end'''

'end'